# 금융통화위원회 의사록 데이터 수집

## 단계1. 데이터를 저장할 폴더로 이동

현재 위치 확인

In [ ]:
pwd

'/content/drive/My Drive/bok'

bok 폴더로 이동

In [ ]:
cd drive/My Drive/bok

/content/drive/My Drive/bok


## 단계2. 금융통화위원회 의사록 pdf 파일 크롤링

크롤링 함수 minutes_crawler 함수 구현 코드

In [ ]:
from bs4 import BeautifulSoup 
import requests
import json
import urllib.request

def minutes_crawler():
  startDate = '2005-05-01'
  endDate = '2017-12-31'
  siteUrl = 'https://www.bok.or.kr'
  page = 0

  while True:
    page += 1
    res = requests.get('https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&searchWrd=&searchCnd=1&sdate={}&edate={}&pageIndex={}'.format(startDate, endDate, page))
    bs = BeautifulSoup(res.text, 'html.parser')
    
    # 요청 페이지 범위를 넘어가면 함수를 종료한다.
    done = bs.select('body div.bdLine.type2 ul li.nofile') 
    if done:
      return
    
    #금통위 의사록 pdf파일 크롤링
    for i in bs.select('body div.container.subContainer div#contentScTop main#content div.bdLine.type2 ul li div.row span.col.m10.s10.x9.ctBx a'):
      listUrl = siteUrl + i.attrs['href']
      title = i.select_one('span.titlesub').text
      res2 = requests.get(listUrl)
      bs2 = BeautifulSoup(res2.text, 'html.parser')
      
      for j in bs2.select('body main#content form#board  div.viewLine dl.down dd div.addfile ul li'):
        a = j.select_one('a')
        if 'pdf' in a.text:
          fileUrl = siteUrl + a.attrs['href']
          urllib.request.urlretrieve(fileUrl, title+'.pdf')
          print(title)

크롤링 함수 minutes_crawler 실행

In [ ]:
minutes_crawler()

## 단계 3. 원하는 데이터만 추출해서 csv 파일로 저장 (제목, 회의일자, 작성일자, 내용)

bok_minutes.csv 파일에 저장

In [ ]:
pip install tika

In [ ]:
from tika import parser
import glob
import csv
import re

csv_file = open('bok_minutes.csv', mode="w", encoding="utf-8", newline="")
writer=csv.writer(csv_file)
writer.writerow(["제목","회의일자", "작성일자", "내용"])

files = glob.glob('*.pdf')
for file in files:
    with open(file, 'r') as f:
      raw = parser.from_file(file)
      print(raw)
      content = raw['content'].replace('\n',' ')
      title = re.sub('\([0-9]{4}.[0-9]{1,2}.[0-9]{1,2}.?\)','',f.name).replace('.pdf','')
      meeting_date = re.findall('[0-9]{4}.[0-9]{1,2}.[0-9]{1,2}', f.name)[0].replace('.','-')
      save_date = re.sub('T.*','',raw['metadata']['Last-Save-Date'])

      writer.writerow([title, meeting_date, save_date, content])
      print(title, meeting_date, save_date)
      print(content)

csv_file.close()


      

csv 파일 panas로 열기

In [ ]:
import numpy as np
import pandas as pd
pd.read_csv('bok_minutes.csv')

,제목,회의일자,작성일자,내용
0,금융통화위원회 의사록(2017년도 제23차),2017-12-7,2017-12-20,...
1,금융통화위원회 의사록(2017년도 제22차),2017-11-30,2017-12-19,...
2,금융통화위원회 의사록(2017년도 제21차),2017-11-9,2017-11-27,...
3,금융통화위원회 의사록(2017년도 제19차),2017-10-19,2017-11-07,...
4,금융통화위원회 의사록(2017년도 제18차),2017-9-21,2017-10-11,...
...,...,...,...,...
257,금융통화위원회 의사록(2005년도 제19차),2005-9-8,2005-10-25,13...
258,금융통화위원회 의사록(2005년도 제17차),2005-8-11,2005-09-27,13...
259,금융통화위원회 의사록(2005년도 제14차),2005-7-7,2005-08-23,13...
260,금융통화위원회 의사록(2005년도 제13차),2005-6-23,2005-08-09,13...
